In [ ]:
!pip uninstall -y serpapi -q
!pip install -q google-search-results google-generativeai


import google.generativeai as genai
from serpapi import GoogleSearch


GOOGLE_API_KEY = "YOUR_GOOGLE_GEMINI_API_KEY"
SERPAPI_API_KEY = "YOUR_SERPAPI_API_KEY"


genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-pro")


conversation_history = []


def chat_with_gemini(user_input):
    conversation_history.append({"role": "user", "parts": [user_input]})
    response = model.generate_content(conversation_history)
    conversation_history.append({"role": "model", "parts": [response.text]})
    return response.text


def search_google(query):
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPAPI_API_KEY,
    }

    search = GoogleSearch(params)
    result = search.get_dict()

    if "answer_box" in result:
        box = result["answer_box"]
        if "answer" in box:
            return box["answer"]
        elif "snippet" in box:
            return box["snippet"]

    if "organic_results" in result and len(result["organic_results"]) > 0:
        return result["organic_results"][0].get("snippet", "No snippet found.")

    return "No relevant data found."


def handle_query(user_input):
    real_time_keywords = ["price", "weather", "time", "today", "now", "news"]

    if any(keyword in user_input.lower() for keyword in real_time_keywords):
        real_time_response = search_google(user_input)
        return f"📡 Real-Time Info: {real_time_response}"
    else:
        return chat_with_gemini(user_input)


def interactive_chat():
    print("🤖 Gemini AI Assistant with Real-Time Web Search")
    print("Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = handle_query(user_input)
        print(f"Gemini: {response}\n")

# To run it in notebooks or scripts:
# interactive_chat()
